In [8]:
# paths to collection config
REVIEWS_PATH = 'reviews'
PRODUCTS_PATH = 'products'

# collection names
REVIEWS_COLLECTION = 'reviews'
PRODUCTS_COLLECTION = 'products'

create_collection(REVIEWS_PATH, REVIEWS_COLLECTION)
create_collection(PRODUCTS_PATH, PRODUCTS_COLLECTION)

In [9]:
import os
import copy

def read_file(filename):
    with open(filename, encoding='utf8') as f:
        return f.read()

def parse_product(line):
    fields = ['asin', 'description', 'title', 'price']
    product1 = eval('(' + line + ')')
    product2 = {}
    if('title' in product1.keys() and len(product1) != 0):
        for field, value in product1.items():
            if(field in fields):
                product2[field] = value
    return product2
    
def product_json(filename):
    list_of_dictionaries = []
    file_text = read_file(filename)
    for line in open(filename):
        product = parse_product(line)
        if(len(product) == 0):
            continue
        list_of_dictionaries.append(product)
    return list_of_dictionaries

# add only reviews with product
def check_asin(reviews1, productsAsin):
    reviews2 = []
    reviewsAsin = copy.deepcopy(list(reviews1.keys()))
    reviewsAsin.sort()
    productsAsin.sort()
    r = 0
    p = 0
    while(r != len(reviewsAsin) and p != len(productsAsin)):
        if(reviewsAsin[r] == productsAsin[p]):
            reviews2 += reviews1[reviewsAsin[r]]
            r += 1
            p += 1
        elif(reviewsAsin[r] < productsAsin[p]):
            r += 1
        else:
            p += 1
    return reviews2

def get_products_asin(products):
    asin = []
    for dict_product in products:
        asin.append(dict_product['asin'])
    return asin

def parse_review(line):
    fields = ['id', 'asin', 'reviewerName', 'reviewText', 'summary', 'overall']
    review1 = eval('(' + line + ')')
    review2 = {}
    if(len(review1) != 0): # no empty lines
        for field, value in review1.items():
            if(field in fields):
                if(field == 'overall'):
                    value = int(value) # trunk to int
                review2[field] = value
    return review2
    
def review_json(filename, products):
    temp = {} # dictionary of asin and reviews with that asin. asin -> [{}, {}, ...]
    file_text = read_file(filename)
    for line in open(filename):
        review = parse_review(line)
        if(review['asin'] in  temp.keys() ):
            temp[review['asin']].append(review)
        else:
            temp[review['asin']] = [review]
    list_of_dictionaries = check_asin(temp, get_products_asin(products))
    return list_of_dictionaries

In [10]:
import pysolr

def solr_call(dictionary, collection):
    solr = pysolr.Solr(f'http://localhost:8983/solr/{collection}', timeout=120)
    solr.add(dictionary, commit=True)
    return None

def index_documents(product_filename, review_filename):
    product_dictionary = product_json(product_filename)
    solr_call(product_dictionary, PRODUCTS_COLLECTION)
    
    reviews_dictionary = review_json(review_filename, product_dictionary)
    solr_call(reviews_dictionary, REVIEWS_COLLECTION)
    return None

Instructions to run the homework

1. Start solr server
2. Launch Jupyter Notebook
3. Set "SOLR_EXECUTABLE" to your own solr bin path
4. Set these contants: 
    1. REVIEWS_PATH and PRODUCTS_PATH with the conf. collection folders
    2. REVIEWS_COLLECTION and PRODUCTS_COLLECTION with the collection names (used on index_document() as well) 
4. Run the Assignment2.ipynb to create reviews and products collections

How to run flask
5. Go to reviewsite folder:
    1. type command line "set FLASK_APP=app.py"
    2. type command line "set FLASK_DEBUG=1"
    3. type command line "flask run"
9. Go to http://127.0.0.1:5000/ and start search